## Pydeck
Pydeck is a high-level Python wrapper for Deck.gl, a powerful WebGL-powered library for visual exploratory data analysis of large datasets. Pydeck is built on top of Deck.gl and provides a simple interface to create complex visualizations with minimal code.

### Advanced Visualization Effects in Pydeck 0.9.1

Pydeck 0.9.1 introduces enhanced lighting, shading, and animation effects, making geospatial visualizations more realistic and interactive. These improvements allow users to create more immersive 3D visualizations, particularly for applications like urban modeling, environmental simulations, and infrastructure planning.

In [ ]:
"""
TextLayer
========

Names of various public transit stops within Leipzig plotted at the location of that stop
"""
# !pip install pydeck
import pydeck as pdk
from pydeck.types import String
import pandas as pd

TEXT_LAYER_DATA = "/Users/ulrike_imac_air/projects/maps/pydeck/LeipzigStops.json"  # noqa
df = pd.read_json(TEXT_LAYER_DATA)

# Define a layer to display on a map
layer = pdk.Layer(
    "TextLayer",
    df,
    pickable=True,
    get_position="coordinates",
    get_text="name",
    get_size=8,
    get_color=[0, 0, 0],
    get_angle=0,
    # Note that string constants in pydeck are explicitly passed as strings
    # This distinguishes them from columns in a data set
    get_text_anchor=String("middle"),
    get_alignment_baseline=String("center"),
)

# Set the viewport location
view_state = pdk.ViewState(latitude=51.345693, longitude=12.380429, zoom=10, bearing=0, pitch=45)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{name}\n{address}"},
    map_style=pdk.map_styles.CARTO_LIGHT,
    # map_style=pdk.map_styles.road,
)

r.show()
r.to_html("text_layerLeipzig.html")

### Expanded Compatibility with Pandas and GeoPandas in Pydeck 0.9.1

Working with geographical data stored in dataframes is made simpler with Pydeck 0.9.1, which improves interoperability with Pandas and GeoPandas. For data scientists and GIS specialists using Python-based geospatial analysis, this upgrade simplifies the process of turning tabular spatial data into interactive maps.

In [3]:
"""
PathLayer
=========

Locations of the public transport lines.
"""

import pandas as pd
import pydeck as pdk

DATA_URL = "/Users/ulrike_imac_air/projects/maps/pydeck/LeipzigStops.json"
df = pd.read_json(DATA_URL)

def hex_to_rgb(h):
    h = h.lstrip("#")
    return tuple(int(h[i : i + 2], 16) for i in (0, 2, 4))

df["color"] = df["color"].apply(hex_to_rgb)


view_state = pdk.ViewState(latitude=51.345693, longitude=12.380429, zoom=9)

layer = pdk.Layer(
    type="PathLayer",
    data=df,
    pickable=True,
    get_color="color",
    width_scale=20,
    width_min_pixels=2,
    get_path="path",
    get_width=5,
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{name}"}, map_style=pdk.map_styles.CARTO_DARK)
r.show()
r.to_html("path_layerLeipzig.html")

KeyError: 'color'